In [1]:
import gc
gc.collect()

import sys
sys.path.insert(0, '../')
import logging
logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
np.random.seed(42)
from datetime import datetime, timedelta
from time import time

from cryptotrader.exchange_api.poloniex import Poloniex
from cryptotrader.envs.trading import BacktestDataFeed, BacktestEnvironment
from cryptotrader.envs.utils import convert_and_clean, convert_to
from cryptotrader.models import cn_models
from cryptotrader.utils import array_normalize

import chainer as cn

import matplotlib.pyplot as plt
from bokeh.io import output_notebook
from jupyterthemes import jtplot
output_notebook()
jtplot.style()
%matplotlib inline
%load_ext line_profiler

Loading BokehJS ...

In [2]:
obs_steps = 50
period = 30
pairs = ["USDT_BTC", "USDT_ETH", "USDT_LTC", "USDT_XRP"]
init_funds = {"BTC":'0.00000000', "ETH":'0.00000000', "LTC":'0.00000000', "XRP":'1.00000000', "USDT":'1.00000000'}

tapi = Poloniex()
tapi = BacktestDataFeed(tapi, period, pairs, init_funds)
tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=100)),
                       start=datetime.timestamp(datetime.utcnow() - timedelta(days=150)))

env = BacktestEnvironment(period, obs_steps, tapi, 'backtest_env')
env.add_pairs(pairs)
env.fiat = 'USDT'
obs = env.reset()

tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=50)),
                       start=datetime.timestamp(datetime.utcnow() - timedelta(days=100)))

test_env = BacktestEnvironment(period, obs_steps, tapi, 'backtest_env')
test_env.add_pairs(pairs)
test_env.fiat = 'USDT'
test_env.reset(); 

INFO:backtest_env:[Trading Environment initialization]
Trading Environment Initialized!

INFO:backtest_env:[Trading Environment initialization]
Trading Environment Initialized!



In [3]:
eiie = cn_models.EIIE(env.get_observation(True).shape[0] - 1, 4, 32)

In [4]:
optimizer = cn.optimizers.Adam(alpha=3e-3)
l2_reg = cn.optimizer.WeightDecay(1e-2)
optimizer.setup(eiie)
optimizer.add_hook(l2_reg)

In [6]:
train_epochs = 1000000
test_interval = 2000
test_epochs = 256
batch_size = 8
save_dir = './save/'

# cn.serializers.load_npz(save_dir + 'eiie.npz', obj=eiie)

cn_models.train_nn(eiie, env, test_env, optimizer, batch_size, train_epochs, test_interval, test_epochs, save_dir, 'eiie')

Training epoch 5/1000000, loss: 0.45923206, r2: -6363.734375, samples/sec: 5.3950649
Interrupted by the user. Best score: -inf
